<a href="https://colab.research.google.com/github/SnehaGhosh10/AI-career-advisor/blob/main/AI_powered_Resume_Matcher_%26_Career_Growth_Advisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymupdf langchain groq


In [ ]:
!pip install -q python-dotenv

from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import os
# Replace with your actual GROQ API key
os.environ["GROQ_API_KEY"] = groq_key

In [ ]:
!pip install -q langchain-groq

In [ ]:
from langchain_groq import ChatGroq

llm_groq = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.1
)


In [ ]:
!pip install -U langchain-community

In [ ]:
from langchain.document_loaders import PyMuPDFLoader

def load_pdf_text(pdf_path):
    loader = PyMuPDFLoader(pdf_path)
    return loader.load()[0].page_content

In [ ]:
resume_text = load_pdf_text("Sneha_Ghosh_Resume.pdf")
jd_text = load_pdf_text("job_description_ai_ml.pdf")

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain.prompts import PromptTemplate

resume_prompt = PromptTemplate(
    input_variables=["document"],
    template="""
You are an intelligent AI Resume Parser.

Your task is to extract structured information from the candidate's resume.
Strictly follow the output format and ensure the JSON is valid.

From the resume content below, extract:

1. Full Name
2. Email Address
3. Skills → Return as a list of strings
4. Work Experience → A list of objects with:
    - job_title
    - company_name
    - duration (e.g., "Jan 2022 – May 2023")
5. Education → A list of objects with:
    - degree
    - university
    - years (e.g., "2022–2026")

If a field is missing, return an empty string or empty list.

 Important:
- Return only valid JSON.
- Do NOT include any extra commentary or markdown formatting.
- Be concise but accurate.

Resume Text:
{document}
"""
)


resume_chain = LLMChain(llm=llm_groq, prompt=resume_prompt)
resume_json = resume_chain.run(document=resume_text)
print(resume_json)

Here is the extracted information in the required format:

```
{
  "Full Name": "Sneha Ghosh",
  "Email Address": "ghoshsneha203@gmail.com",
  "Skills": [
    "Python",
    "Java",
    "SQL",
    "Pandas",
    "NumPy",
    "Matplotlib",
    "Excel",
    "DBMS",
    "MySQL",
    "Scikit-learn",
    "TensorFlow",
    "Keras"
  ],
  "Work Experience": [
    {
      "job_title": null,
      "company_name": null,
      "duration": null
    }
  ],
  "Education": [
    {
      "degree": "BTech",
      "university": "University of Engineering and management kolkata",
      "years": "AUG 2022 – JULY 2026"
    }
  ]
}
```

Note that the "Work Experience" section is empty because there is no relevant information in the resume.


In [ ]:
jd_prompt = PromptTemplate(
    input_variables=["document"],
    template="""
You are a smart AI Job Description (JD) parser.

Extract structured information from the following job description text.

Return a valid JSON with the following keys:

1. "role_title" → Job title (e.g., "AI/ML Engineer", "Data Analyst")
2. "required_skills" → List of core technical skills (e.g., ["Python", "SQL", "TensorFlow"])
3. "responsibilities" → List of key responsibilities (concise bullet points)
4. "preferred_experience" → List of qualifications or years of experience required or preferred

Rules:
- Use only the text provided.
- Do NOT infer or guess.
- Do NOT include explanations or markdown.
- Return a clean, valid JSON only.

Job Description:
{document}
"""
)


jd_chain = LLMChain(llm=llm_groq, prompt=jd_prompt)
jd_json = jd_chain.run(document=jd_text)
print(jd_json)


Here is the extracted JSON:

{
"role_title": "AI/ML Engineer or Data Analyst",
"required_skills": [
"Python",
"Pandas",
"NumPy",
"scikit-learn",
"TensorFlow",
"PyTorch",
"SQL",
"Matplotlib",
"Seaborn",
"cloud platforms like AWS or GCP"
],
"responsibilities": [
"Develop and deploy machine learning models for classification, regression, or clustering tasks.",
"Analyze large datasets to derive actionable insights and build predictive analytics.",
"Collaborate with data engineers to optimize data pipelines.",
"Create dashboards and reports using tools like Tableau or Power BI.",
"Communicate findings to business and technical stakeholders."
],
"preferred_experience": [
"2+ years of experience in data science or machine learning roles.",
"Master's degree in Computer Science, Data Science, or related field.",
"Knowledge of MLOps practices and deployment strategies."
]
}


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

compare_prompt = PromptTemplate(
    input_variables=["resume_json", "jd_json"],
    template="""
You are an expert AI Career Advisor.

Your job is to evaluate how well a candidate's resume matches a specific job description.

Carefully analyze the following JSON inputs:

- Resume JSON: {resume_json}
- Job Description JSON: {jd_json}

Evaluation Criteria:
1. Skills Match: Compare technical and soft skills.
2. Experience Alignment: Check if work experience matches JD expectations (years, tools, domain).
3. Education Relevance: Evaluate if educational background meets the JD criteria.

Your Tasks:
- Calculate an overall `match_score` (0–100), based on skills, experience, and education.
- Identify `missing_skills` that are in the JD but not in the resume.
- Provide 2–4 actionable `recommendations` to improve the resume (e.g., add projects, list tools, get certifications).
- Write personalized `feedback` summarizing strengths and gaps (2–4 sentences).

Return ONLY a valid JSON object in this format:
{{
  "match_score": 0-100,
  "missing_skills": ["skill1", "skill2", ...],
  "recommendations": ["tip1", "tip2", ...],
  "feedback": "Short professional summary (2–4 sentences)"
}}

Rules:
- DO NOT include markdown, explanation, or extra text.
- If all skills match, `missing_skills` should be an empty list.
- Be realistic and helpful, not overly harsh or generic.
"""
)


compare_chain = LLMChain(llm=llm_groq, prompt=compare_prompt)
comparison_json = compare_chain.run(resume_json=resume_json, jd_json=jd_json)
print(comparison_json)

Here is the evaluation result:

```
{
  "match_score": 60,
  "missing_skills": ["PyTorch", "Seaborn", "cloud platforms like AWS or GCP"],
  "recommendations": ["Add projects showcasing AI/ML skills", "List specific tools used in work experience", "Consider obtaining certifications in AI/ML or data science"],
  "feedback": "Sneha has a strong foundation in AI/ML skills, but lacks experience and certifications. She should focus on building a portfolio of projects and highlighting relevant tools to increase her chances of getting hired."
}
```


In [ ]:
import json
import re


json_match = re.search(r'\{.*\}', comparison_json, re.DOTALL)

if json_match:
    json_string = json_match.group(0)
    parsed = json.loads(json_string)

    missing_skills = parsed.get("missing_skills", [])
    learning_path = [f"Take a course on {skill} (e.g., Coursera/Udemy)" for skill in missing_skills]

    parsed["learning_path"] = learning_path

    import pprint
    pprint.pprint(parsed)
else:
    print("Could not find a JSON object in the output.")

{'feedback': 'Sneha has a strong foundation in AI/ML skills, but lacks '
             'experience and certifications. She should focus on building a '
             'portfolio of projects and highlighting relevant tools to '
             'increase her chances of getting hired.',
 'learning_path': ['Take a course on PyTorch (e.g., Coursera/Udemy)',
                   'Take a course on Seaborn (e.g., Coursera/Udemy)',
                   'Take a course on cloud platforms like AWS or GCP (e.g., '
                   'Coursera/Udemy)'],
 'match_score': 60,
 'missing_skills': ['PyTorch', 'Seaborn', 'cloud platforms like AWS or GCP'],
 'recommendations': ['Add projects showcasing AI/ML skills',
                     'List specific tools used in work experience',
                     'Consider obtaining certifications in AI/ML or data '
                     'science']}


In [ ]:
with open("career_advice_report.json", "w") as f:
    json.dump(parsed, f, indent=2)


In [ ]:
from google.colab import files
files.download("career_advice_report.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>